# Creating dataset for S&P 500 stocks

__Adding Industry Tag for each company in the S&P 500 for each trading day__

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
import datetime


# Specify the date range
start_date = '2016-01-01'
end_date = '2020-04-02'

# Step 1: Get a list of ticker symbols for companies in the S&P500 index
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
sectors = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text.strip()
    sector = row.findAll('td')[2].text.strip()
    tickers.append(ticker)
    sectors.append(sector)


# Define date range
start_date = datetime.datetime(2016, 1, 1)
end_date = datetime.datetime(2020, 4, 2)

# Define S&P500 tickers
tickers_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tickers_table = pd.read_html(tickers_url, header=0)[0]
tickers = tickers_table['Symbol'].tolist()

# Define dataframe to hold stock prices
prices_df = pd.DataFrame(columns=['Date', 'Ticker', 'Open', 'Low', 'High', 'Close', 'Volume', 'Sector'])

# Loop over tickers and retrieve stock prices
for ticker in tickers:
    try:
        # Retrieve data from Yahoo Finance API
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data.reset_index(inplace=True)
        stock_data['Ticker'] = ticker
        
        # Extract relevant columns
        columns = ['Date', 'Ticker', 'Open', 'Low', 'High', 'Close', 'Volume'] # TODO add adj Close
        stock_data = stock_data[columns]
        
        # Add sector information
        sector = tickers_table[tickers_table['Symbol'] == ticker]['GICS Sector'].iloc[0]
        stock_data['Sector'] = sector
        
        # Append to prices dataframe
        prices_df = pd.concat([prices_df, stock_data])
        
        print(f'{ticker} data retrieved')
    except Exception as e:
        print(f'{ticker} error: {e}')

# Save dataframe to CSV
prices_df.to_csv('SP500_stock_prices.csv', index=False)



[*********************100%***********************]  1 of 1 completed
MMM data retrieved
[*********************100%***********************]  1 of 1 completed
AOS data retrieved
[*********************100%***********************]  1 of 1 completed
ABT data retrieved
[*********************100%***********************]  1 of 1 completed
ABBV data retrieved
[*********************100%***********************]  1 of 1 completed
ACN data retrieved
[*********************100%***********************]  1 of 1 completed
ATVI data retrieved
[*********************100%***********************]  1 of 1 completed
ADM data retrieved
[*********************100%***********************]  1 of 1 completed
ADBE data retrieved
[*********************100%***********************]  1 of 1 completed
ADP data retrieved
[*********************100%***********************]  1 of 1 completed
AAP data retrieved
[*********************100%***********************]  1 of 1 completed
AES data retrieved
[*********************100%***

__Importing EPS for each day__

In [2]:
# import Webscraping libraries
# pandas, time, BeautifulSoup, webdriver

import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [19]:
# downlaod chrome driver from https://chromedriver.chromium.org/downloads
# Setup chrome driver (direct it to your path) Dont close the window popup
path = 'C:\Program Files (x86)\chromedriver\chromedriver.exe'
driver = webdriver.Chrome(path)

C:\Users\nicol\AppData\Local\Temp\ipykernel_2960\3118529260.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [4]:
# Static variables

# Retrieve the individual stocks of the sp500 from wikipedia and create a list of tickers
# Also retrieve the GICS sector of each stock for later use
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# Read the table of components from the Wikipedia page
sp500_components = pd.read_html(url)[0]
# Extract the ticker and sector symbols from their columns
tickers = sp500_components["Symbol"].tolist()
sectors = sp500_components["GICS Sector"].tolist()

In [5]:
# Macrotrends url to scrape
# driver.get retrieves the url and opens the page - it is from driver. that we get functionality on the site
def setup_url_and_driver(ticker):
    url = f'https://www.macrotrends.net/stocks/charts/{ticker}/3m/pe-ratio'
    driver.get(url)


In [20]:
df_list = []
# Loop through each ticker in the tickers list
for ticker in tickers:
    # Set up the URL and driver for the ticker
    setup_url_and_driver(ticker)
    
    try:
        # Extract the table data from the webpage
        table_html = driver.find_elements(by=By.CLASS_NAME, value="table")[0].get_attribute('outerHTML')
        table = pd.read_html(table_html, parse_dates=True, skiprows=1, header=0)[0]
        table.sort_values(by='Date', inplace=True, ascending=True)


        # Convert the 'date' column to datetime format
        table['Date'] = pd.to_datetime(table['Date'])

        # Set 'date' column as the index of the DataFrame
        table.set_index('Date', inplace=True)

        # Define the time period you want to extract
        start_date = '2016-1-1'
        end_date = '2020-4-2'

        # Use boolean indexing to select the rows within the time period
        df_period = table.loc[(table.index >= start_date) & (table.index <= end_date)]

        # Filter out all columns except 'TTM NET EPS'
        df_period = df_period[['TTM Net EPS']]

        # Add a column for the ticker symbol
        df_period['Ticker'] = ticker

        # Append the resulting DataFrame to a list
        df_list.append(df_period)
    except:
        print(f"Error for {ticker}. Skipping...")

# Concatenate all DataFrames in the list into a single DataFrame
result_df = pd.concat(df_list)


In [21]:
result_df.to_csv('quarterly_EPS.csv', index=True)


__Add filler days in between the quarterly earning numbers__

In [43]:
import pandas as pd
import numpy as np

# Read in the quarterly EPS data
eps_data = pd.read_csv('quarterly_EPS.csv')

# Convert the 'Date' column to a datetime type
eps_data['Date'] = pd.to_datetime(eps_data['Date'])

# Set the 'Date' column as the index
eps_data = eps_data.set_index('Date')

# Create a date range from December 31st, 2022 to March 15th, 2023
date_range = pd.date_range('2022-12-31', '2023-03-15', freq='D')

# Create an empty dataframe to store the final data
final_eps_data = pd.DataFrame()

# Loop over the unique tickers
for ticker in eps_data['Ticker'].unique():
    # Filter the EPS data for the current ticker
    ticker_eps_data = eps_data[eps_data['Ticker'] == ticker]
    
    # Resample to a daily frequency and forward fill missing values
    ticker_eps_data = ticker_eps_data.resample('D').ffill()
    
    # Reindex the data to the full date range and forward fill missing values
    ticker_eps_data = ticker_eps_data.reindex(date_range, method='ffill')
    
    # Add the ticker column back to the data
    ticker_eps_data['Ticker'] = ticker
    
    # Append the ticker data to the final data
    final_eps_data = final_eps_data.append(ticker_eps_data)

# Print the final data
print(final_eps_data)



           TTM Net EPS Ticker
2022-12-31      $10.15    MMM
2023-01-01      $10.15    MMM
2023-01-02      $10.15    MMM
2023-01-03      $10.15    MMM
2023-01-04      $10.15    MMM
...                ...    ...
2023-03-11       $4.50    ZTS
2023-03-12       $4.50    ZTS
2023-03-13       $4.50    ZTS
2023-03-14       $4.50    ZTS
2023-03-15       $4.50    ZTS

[37575 rows x 2 columns]


In [41]:
final_eps_data.to_csv('daily_EPS.csv', index=True)

__Add daily EPS values to the stock index:__

In [62]:
# make the final_eps_data a dataframe
final_eps_data = pd.DataFrame(final_eps_data)

# make the date a column
final_eps_data['Date'] = final_eps_data.index

# make the dataframe start from 2023-01-03
final_eps_data = final_eps_data[final_eps_data['Date'] >= '2023-01-03']

# remove the chosen index
final_eps_data = final_eps_data.reset_index(drop=True)


final_eps_data

,TTM Net EPS,Ticker,Date
0,$10.15,MMM,2023-01-03
1,$10.15,MMM,2023-01-04
2,$10.15,MMM,2023-01-05
3,$10.15,MMM,2023-01-06
4,$10.15,MMM,2023-01-07
...,...,...,...
36067,$4.50,ZTS,2023-03-11
36068,$4.50,ZTS,2023-03-12
36069,$4.50,ZTS,2023-03-13
36070,$4.50,ZTS,2023-03-14


In [52]:
prices_df

,Date,Ticker,Open,Low,High,Close,Volume,Sector
0,2023-01-03,MMM,121.519997,120.370003,122.639999,122.470001,2612800,Industrials
1,2023-01-04,MMM,123.349998,122.709999,125.290001,125.150002,2769700,Industrials
2,2023-01-05,MMM,124.209999,122.459999,124.570000,122.959999,2606600,Industrials
3,2023-01-06,MMM,124.660004,123.750000,127.129997,126.720001,2417000,Industrials
4,2023-01-09,MMM,127.000000,126.110001,129.460007,126.790001,2871300,Industrials
...,...,...,...,...,...,...,...,...
24544,2023-03-08,ZTS,166.800003,166.699997,169.130005,168.639999,960100,Health Care
24545,2023-03-09,ZTS,168.889999,167.130005,170.440002,167.490005,1527500,Health Care
24546,2023-03-10,ZTS,163.000000,160.070007,163.360001,161.529999,3326400,Health Care
24547,2023-03-13,ZTS,161.990005,159.850006,164.889999,163.410004,2772100,Health Care


In [73]:
import pandas as pd

# read in the two dataframes
prices_df = pd.read_csv('SP500_stock_prices.csv')
ttm_eps_df = final_eps_data

# convert the "Date" column in the prices_df dataframe to datetime type
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

# merge the two dataframes on "Date" and "Ticker"
prices_df = pd.merge(prices_df, ttm_eps_df, on=["Date", "Ticker"])




__Adding PE-ratio to the data__

In [74]:
# add column for the PE-ratio, which is the price divided by the TTM Net EPS
prices_df['TTM Net EPS'] = prices_df['TTM Net EPS'].str.replace('$','').astype(float)
prices_df['PE-Ratio'] = prices_df['Close'] / prices_df['TTM Net EPS']
prices_df.drop('TTM Net EPS', axis=1, inplace=True)
prices_df.index = pd.to_datetime(prices_df.index)

# remove index from dataframe
prices_df = prices_df.reset_index(drop=True)
# Make the date column the new index
prices_df = prices_df.set_index('Date')

prices_df.to_csv('SP500_stock_dataset_PEandMarketCap.csv', index=True)


C:\Users\nicol\AppData\Local\Temp\ipykernel_2960\1001243099.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  prices_df['TTM Net EPS'] = prices_df['TTM Net EPS'].str.replace('$','').astype(float)


In [75]:
prices_df

,Ticker,Open,Low,High,Close,Volume,Sector,PE-Ratio
Date,,,,,,,,
2023-01-03,MMM,121.519997,120.370003,122.639999,122.470001,2612800,Industrials,12.066010
2023-01-04,MMM,123.349998,122.709999,125.290001,125.150002,2769700,Industrials,12.330049
2023-01-05,MMM,124.209999,122.459999,124.570000,122.959999,2606600,Industrials,12.114286
2023-01-06,MMM,124.660004,123.750000,127.129997,126.720001,2417000,Industrials,12.484729
2023-01-09,MMM,127.000000,126.110001,129.460007,126.790001,2871300,Industrials,12.491626
...,...,...,...,...,...,...,...,...
2023-03-08,ZTS,166.800003,166.699997,169.130005,168.639999,960100,Health Care,37.475555
2023-03-09,ZTS,168.889999,167.130005,170.440002,167.490005,1527500,Health Care,37.220001
2023-03-10,ZTS,163.000000,160.070007,163.360001,161.529999,3326400,Health Care,35.895555


Adding Market Cap - Values

In [76]:
# import webscraping libraries
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import requests


# Retrieve the individual stocks of the S&P 500 from Wikipedia and create a list of tickers
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_components = pd.read_html(url)[0]
tickers = sp500_components["Symbol"].tolist()
tickers = tickers[0:5] # for testing purposes

# Create an empty DataFrame to store all ticker tables
combined_table = pd.DataFrame()

# Function to scrape data for a single ticker and append to the combined table
def scrape_data_for_ticker(ticker):
    # Setup chrome driver
    path = 'C:\Program Files (x86)\chromedriver\chromedriver.exe'
    driver = webdriver.Chrome(path)
    # Macrotrends url to scrape
    url = f'https://ycharts.com/companies/{ticker}/market_cap'
    driver.get(url)

   



# Set the headers to simulate a user agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Create an empty list to store the data for each ticker
data = []

# Loop through tickers list and scrape data for each ticker
for ticker in tickers:
    # Build the URL for the ticker
    url = f"https://ycharts.com/companies/{ticker}/market_cap"
    
    # Send a GET request to the URL with the headers
    response = requests.get(url, headers=headers)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the first table element by its class name
    table1 = soup.find('table', {'class': 'table'})
    
    # Find the second table element by its class name
    table2 = soup.find_all('table', {'class': 'table'})[1]
    
    # Convert the table HTML to pandas dataframes
    df1 = pd.read_html(str(table1))[0]
    df2 = pd.read_html(str(table2))[0]
    
    # Combine the two dataframes into a single one
    df = pd.concat([df1, df2], axis=0)
    
    # Add the ticker name as a new column
    df['Ticker'] = ticker
    # Make the 'Date' column the index of the DataFrame
    df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)
    

    # Sort the DataFrame by 'Ticker' and 'Date'
    df = df.sort_values(by='Date', ascending=True)
    # Append the dataframe to the list of dataframes
    data.append(df)
    



# Combine the data for all tickers into a single dataframe
combined_table = pd.concat(data, axis=0)


# Change Value name to Market Cap
combined_table.rename(columns={'Value': 'Market Cap'}, inplace=True) 

# For T, B and M, convert to float
combined_table['Market Cap'] = combined_table['Market Cap'].apply(lambda x: float(x[:-1]) * 1000000000000 if x[-1] == 'T' else float(x[:-1]) * 1000000 if x[-1] == 'M' else float(x[:-1]) * 1000000000 if x[-1] == 'B' else float(x[:-1]))

print(combined_table)
# Save the combined table to a single csv file
combined_table.to_csv("combined_market_cap.csv")

              Market Cap Ticker
Date                           
2023-01-20  6.627000e+10    MMM
2023-01-23  6.735000e+10    MMM
2023-01-24  6.316000e+10    MMM
2023-01-25  6.203000e+10    MMM
2023-01-26  6.237000e+10    MMM
...                  ...    ...
2023-03-27  1.815200e+11    ACN
2023-03-28  1.822100e+11    ACN
2023-03-29  1.854900e+11    ACN
2023-03-30  1.872900e+11    ACN
2023-03-31  1.893800e+11    ACN

[250 rows x 2 columns]


In [ ]:

MC = pd.read_csv('combined_market_cap.csv')
prices_df = pd.read_csv('SP500_stock_dataset_PEandMarketCap.csv')

# Add TTM Net EPS to the sp500_stock_prices dataframe
prices_df = pd.merge(prices_df, MC, how='left', on=['Date', 'Ticker'])
prices_df.set_index('Date', inplace=True)

# Save dataframe to CSV
prices_df.to_csv('SP500_stock_dataset_All.csv', index=True)

In [ ]:
prices_df

,Ticker,Open,Low,High,Close,Volume,Sector,PE-Ratio,Market Cap
Date,,,,,,,,,
2023-01-03,MMM,121.519997,120.370003,122.639999,122.470001,2612800,Industrials,NaN,NaN
2023-01-04,MMM,123.349998,122.709999,125.290001,125.150002,2769700,Industrials,NaN,6.874000e+10
2023-01-05,MMM,124.209999,122.459999,124.570000,122.959999,2606600,Industrials,NaN,6.754000e+10
2023-01-06,MMM,124.660004,123.750000,127.129997,126.720001,2417000,Industrials,NaN,6.960000e+10
2023-01-09,MMM,127.000000,126.110001,129.460007,126.790001,2871300,Industrials,NaN,6.964000e+10
...,...,...,...,...,...,...,...,...,...
2023-03-08,ZTS,166.800003,166.699997,169.130005,168.639999,960100,Health Care,NaN,NaN
2023-03-09,ZTS,168.889999,167.130005,170.440002,167.490005,1527500,Health Care,NaN,NaN
2023-03-10,ZTS,163.000000,160.070007,163.360001,161.529999,3326400,Health Care,NaN,NaN


In [2]:


while True:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By
    import time

    # Start the web driver and open the YouTube video
    driver = webdriver.Chrome()
    driver.get("https://www.youtube.com/watch?v=KV4aVrTAqqs")

    # Wait for the cookie banner to appear and click "Accept"
    cookie_accept = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//button[@aria-label='Accept All']")))
    cookie_accept.click()

    # Wait for the video to load
    time.sleep(10)

    # Play the video
    video_player = driver.find_element_by_xpath("//button[@class='ytp-play-button ytp-button']")
    video_player.click()

    # Wait for the video to finish
    time.sleep(120)

    # Close the driver
    driver.quit()



TimeoutException: Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x0082DCE3]
	(No symbol) [0x007C39D1]
	(No symbol) [0x006D4DA8]
	(No symbol) [0x0070019F]
	(No symbol) [0x007003AB]
	(No symbol) [0x0072EE62]
	(No symbol) [0x0071AF14]
	(No symbol) [0x0072D57C]
	(No symbol) [0x0071ACC6]
	(No symbol) [0x006F6F68]
	(No symbol) [0x006F80CD]
	GetHandleVerifier [0x00AA3832+2506274]
	GetHandleVerifier [0x00AD9794+2727300]
	GetHandleVerifier [0x00ADE36C+2746716]
	GetHandleVerifier [0x008D6690+617600]
	(No symbol) [0x007CC712]
	(No symbol) [0x007D1FF8]
	(No symbol) [0x007D20DB]
	(No symbol) [0x007DC63B]
	BaseThreadInitThunk [0x76A67D69+25]
	RtlInitializeExceptionChain [0x77DFB74B+107]
	RtlClearBits [0x77DFB6CF+191]
